In [1]:
import sciopy as scpy

In [2]:
scpy.save_settings()

Saved Settings: 0x90


In [3]:
scpy.software_reset()

Software reset: 0xA1


In [5]:
scpy.acknowledge_messages()

Communication-frames with incorrect syntax will cause a “Frame-Not-Acknowledge” message
If the transmission of a communication-frame is interrupted for more than 10 ms a “Timeout” message is send
Every invalid command-tag will cause a “Not-Acknowledge” message
Every valid command is acknowledged with an acknowledge command [ACK]
For commands with a return value the returning frame comes before the acknowledge message
When commands are sent during the current measurements, measurement data can be transmitted between the command and the following returning frame and the acknowledge-message (commands are handled asynchronously)
Before sending a new command, the resulting acknowledge or not acknowledge of the previous command has to be awaited.
